# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from karabo_data import RunDirectory

from xas_processor import XasFastADC

In [ ]:
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0150"
run = RunDirectory(run_folder)
run.info()

In [ ]:
xgm_device = 'SCS_BLU_XGM/XGM/DOOCS'

adc_device = 'SCS_UTC1_MCP/ADC/1'
adc_channel_names = ["MCP1", "MCP2", "MCP3", "MCP4"]
adc_channels = {name: "{}:channel_{}.output".format(adc_device, ch) 
                for name, ch in zip(adc_channel_names, [9, 8, 7, 6])}

## Sanity check

*check whether the pulse energy in the pipeline data is calibrated*

In [ ]:
idx = -1
tid, data = run.train_from_index(idx)

In [ ]:
xgm_run = run.get_dataframe(fields=[(xgm_device, '*value')])
xgm_run.rename(columns=lambda x: x.split('/')[-1], inplace=True)

# pulse_energy_train_resolved = xgm_run['pulseEnergy.photonFlux'].iloc[idx]
# pulse_energy_pulse_resolved = data[xgm_device + ':output']['data.intensityTD'][
#     0:int(xgm_run['pulseEnergy.nummberOfBrunches'].iloc[idx])].mean()

# print("Train resolved pulse energy is: {:.4f} micro J".format(pulse_energy_train_resolved))
# print("Pulse resolved pulse energy is: {:.4f} micro J".format(pulse_energy_pulse_resolved))
# if abs(pulse_energy_train_resolved - pulse_energy_pulse_resolved) > 0.01*pulse_energy_train_resolved:
#     print("Warning: pulse resolved pulse energy is not calibrated!")

***check the numbers of pulses and train resolved pulse energies throughtout the run***

In [ ]:
_, ax = plt.subplots(figsize=(12, 4.5))
ax2 = ax.twinx()

ln1 = ax.plot(xgm_run['pulseEnergy.nummberOfBrunches'], label="number of pulses", c='g')
ln2 = ax2.plot(xgm_run['pulseEnergy.photonFlux'], label=r"pulse energy ($\mu$J)")

lns = ln1 + ln2
lables = [l.get_label() for l in lns]
ax.legend(lns, lables, loc=0)
ax.set_xlabel("Train ID")
ax.set_ylabel("Number of pulses")
ax2.set_ylabel(r"Pulse energy ($\mu$J)")

plt.tight_layout()

***check that the interested FastAdc channels all have data***

In [ ]:
fig, axes = plt.subplots(len(adc_channels), 1, figsize=(14, 4.5*len(adc_channels)))

for ax, channel in zip(axes, adc_channels):
    channel_data = data[adc_channels[channel]]
    ax.plot(channel_data['data.rawData'] - channel_data['data.baseline'])
    ax.set_title(channel)
    ax.set_ylabel("intensity (arb.)")

ax.set_xlabel("samples")

print("Trigger time: {}".format(data[adc_device]['triggerTime.value']))
print("Delay: {}".format(data[adc_device]['delay.value']))

## Correlation between I0 and I1

In [ ]:
run_energies = {
     95:704.5,  96:705  ,  97:705.5,  98:706  ,  99:706.5, 100:707  , 101:707.5, 102:708  , 
    103:708.5, 104:709  , 105:709.5, 106:710  , 107:710.5, 108:711  , 109:711.5, 110:712  ,
    111:712.5, 112:713  , 113:713.5, 114:714  , 115:714.5, 116:715  , 117:715.5, 118:716  ,
    119:697  , 120:697.5, 121:698  , 122:698.5, 123:699  , 124:699.5, 125:700  , 126:700.5, 
    127:701  , 128:701.5, 129:702  , 130:702.5, 131:703  , 132:703.5, 133:704  , 134:704.5, 
    135:705  , 136:706  , 137:706.5, 138:707  , 139:707.5, 140:708  , 141:708.5, 142:709  , 
    143:709.5, 144:710  , 145:710.5, 146:711}

In [ ]:
xas = XasFastADC(xgm_id=xgm_device, adc_id=adc_device, adc_channels=adc_channels)

run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0098"
xas.process_run(run_folder)
corrs = []
for name in adc_channel_names:
    # 'XasFastADC.correlation()' returns a panda.DataFrame object
    corrs.append(xas.correlation(name))

corrs[0].head()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, corr, name in zip(np.ravel(axes), corrs, adc_channel_names):
    corr.plot.scatter(x="XGM", y="MCP", ax=ax, alpha=0.05)

    ax.set_xlabel(r"$I_0$ ($\mu$ J)")
    ax.set_ylabel(r"$I_1$ (arb.)")
    ax.set_title(name)

fig.suptitle(run_folder)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

## Absorption spectrum

In [ ]:
xas = XasFastADC(xgm_id=xgm_device, adc_id=adc_device, adc_channels=adc_channels)

# Use case without softmono in SCS DAQ data
for run_number, energy in run_energies.items():
    xas.process_run("/gpfs/exfel/exp/SCS/201831/p900048/raw/r{:04d}".format(run_number), photon_energy=energy)

In [ ]:
# 'XasFastADC.spectrum()' returns a panda.DataFrame object
spectrums = dict()
for name in adc_channel_names:
    spectrums[name] = xas.spectrum(name)

spectrums["MCP1"].head()

In [ ]:
ax = spectrums["MCP1"].plot.scatter(x="photon_energy", y="weight")
ax.set_xlabel("Photon energy (ev)")
ax.set_ylabel(r"Weight ($\mu$J)")

In [ ]:
# plot the "masked" spectrum

selected_channels = ["MCP1", "MCP2", "MCP3"]
colors = ['r', 'g', 'b']

fig, ax = plt.subplots(figsize=(8, 6))
for name, color in zip(selected_channels, colors):
    sp = spectrums[name]
    # masked by applying a lower boundary to weight
    sp[sp['weight'] > 300].plot.scatter(x="photon_energy", y="absorption", s=40, c=color, ax=ax)

ax.set_xlabel("Photon energy (ev)")
ax.set_ylabel("absorption (arb.)")  # -log(I1/I0)    
ax.legend(selected_channels)